## Группа DS03-onl

Студент Парфимович Алексей

## Домашнее задание №14
- Проанализировать данные по выживаемости пассажиров титаника. 
- Обучить модель предсказания выживет ли человек или нет (ключевая переменная) 
- Определить наиболее значимые переменные которые влияют на выживаемость. 
- Визуализировать дерево решений. 
- Написать вывод. 

(различные варианты визуализации можно посмотреть здесь https://www.kaggle.com/competitions/titanic/code?competitionId=3136&sortBy=scoreDescending&language=Python).

Ответить на вопросы (обоснованно):
1. Как влияет размер семьи  на вероятность виживаемости (одинокие выживают чаще или нет?, люди у которых большая семья (больше 4) выживают чаще или нет).
2. Как влияет пол/возраст/номер палубы. 

In [149]:
import re 
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
 
from math import nan
from datetime import datetime 

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import PolynomialFeatures
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB

from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score


TRAIN_CSV_PATH = 'titanic_train.csv'
TEST_CSV_PATH = 'titanic_test.csv'

- PassengerId — id пассажира
- Survived — погиб (0) или выжил (1)
- Pclass — класс билета (первый (1), второй (2) или третий (3))
- Name — имя пассажира
- Sex — пол
- Age — возраст
- SibSp — количество братьев и сестер или супругов на борту
- Parch — количество родителей и детей на борту
- Ticket — номер билета
- Fare — стоимость билета
- Cabin — номер каюты
- Embarked — порт посадки (C — Шербур; Q — Квинстаун; S — Саутгемптон)

In [150]:
def load_data(file_name:str, cols:list):
    df = pd.read_csv(file_name, usecols = cols, sep=',')

    # Привести все строковые значения к единому регистру, 
    df = df.apply(lambda x: x.astype(str).str.lower() if x.dtype.name=='object' else x)

    # Заменить значения "nan" на NULL/NAN
    df.replace('nan', nan, inplace=True)

    # Выделить из признака "Cabin"(номер каюты) палубу "Deck"
    df.insert(10,'Deck', df['Cabin'].str.extract(r'(\D+)', expand = True)[0])

    # Выделить из признака Name новый признак Title(титул-приставка к имени пассажира)
    df.insert(2,'Title', df['Name'].str.extract(r' (\w+)\.', expand = True)[0] )

    # Выделить признак количества пассажиров зарегистрированных по одному билету
    df.insert(8,'Persons_per_ticket', df.groupby('Ticket')['Fare'].transform('count'))
    # Выделить признак стоимости одного проездного места в билете
    df.insert(9,'Price_per_person', (df['Fare'] / df['Persons_per_ticket']).round(decimals=2))

    # Удалить признаки 'Name','Ticket','Fare','Cabin' как не несущие существенной информации
    df.drop(['Name','Ticket','Fare','Cabin'], inplace=True, axis=1)

    # Для признака "Age" - заменить пропущенные значения наиболее частым значением из группы по признакам 'Title'(титул), и 'Parch'(Наличие родителей/детей)
    df['Age'] = df.groupby(['Title','Parch'])['Age'].transform(lambda x: x.fillna(x.mode()[0] if x.count()>0 else nan))

    # Для признака "Embarked" - заменить пропущенные значения наиболее частым значением данного признака
    df['Embarked'].fillna(df.mode()['Embarked'][0], inplace=True)

    return df

Загрузка и обработка обучающего набора данных

In [155]:
cols = ['Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']

# Загрузить обучающий набор данных
df_train = load_data(TRAIN_CSV_PATH, cols)

num_cols = ['Age','SibSp','Parch','Persons_per_ticket','Price_per_person']
cat_cols = ['Survived','Pclass', 'Title', 'Sex', 'Deck', 'Embarked']

df_num = df_train[num_cols]
df_dummed = pd.get_dummies(df_train[cat_cols])

data = pd.concat((df_num, df_dummed), axis=1)

y_train = data['Survived']
X_train = data.drop(['Survived'], axis=1)

Загрузка и обработка тестового набора данных

In [156]:
cols = ['Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']

# Загрузить тестовый набор данных
df_test = load_data(TEST_CSV_PATH, cols)

num_columns = ['Age','SibSp','Parch','Persons_per_ticket','Price_per_person']
cat_columns = ['Pclass', 'Title', 'Sex', 'Deck', 'Embarked']

df_num = df_test[num_columns]
df_dummed = pd.get_dummies(df_test[cat_columns])

X_test = pd.concat((df_num, df_dummed), axis=1)

In [159]:
forest = (RandomForestClassifier)()
                                
forest.fit(X_train, y_train);

print('AUC на обучающей выборке: {:.3f}'.
      format(roc_auc_score(y_train, forest.predict_proba(X_train)[:, 1])))
#print('AUC на контрольной выборке: {:.3f}'.
#      format(roc_auc_score(y_test, forest.predict_proba(X_test)[:, 1])))

AUC на обучающей выборке: 0.997
